# Organización del número de contratos por municipio

## Limpieza de los datos

In [3]:
import os

def clean_and_save_to_new_folder(input_folder, output_folder):
    # Crear la carpeta de salida si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Obtener la lista de archivos CSV en la carpeta de entrada
    archivos_csv = [archivo for archivo in os.listdir(input_folder) if archivo.endswith('.csv')]

    # Datos excepcionales
    unuseful_words = ['actividad', 'Serie;;;']
    headers = "Serie;Codigo_municipio;Nombre;2020;2021;2022;2023\n"

    # Limpiar y guardar los archivos CSV en la carpeta de salida
    for archivo in archivos_csv:
        archivo_original = os.path.join(input_folder, archivo)
        archivo_limpiado = os.path.join(output_folder, archivo)
        with open(archivo_original, 'r') as f_input, open(archivo_limpiado, 'w') as f_output:
            f_output.write(headers) 
            delete_next = False
            for line in f_input:                    
                if not delete_next:
                    delete = False
                    for word in unuseful_words:
                        if word in line:
                            delete = True
                    if line.strip() == '':
                        delete = False
                    elif '-' in line:
                        line = line.replace('-', '0')
                    if not delete:
                        fields = line.strip().split(';')
                        f_output.write(';'.join(fields[:3] + fields[-5:]) + '\n')
                    if 'Zarzalejo' in line:
                        delete_next = True # Eliminar filas posteriores

# Rutas de las carpetas de entrada y salida
input_folder = ".\original_data"
output_folder = ".\clean_data"

# Limpiar y guardar los archivos CSV en la nueva carpeta
clean_and_save_to_new_folder(input_folder, output_folder)


## Unión de los datos

In [14]:
import os
import csv

def unir_datos(folder_path, output_path):
    # Crear un diccionario para almacenar los datos unidos
    datos_unidos = {}

    # Obtener la lista de archivos en la carpeta
    archivos = os.listdir(folder_path)

    # Iterar sobre cada archivo en la carpeta
    campos_finales = [] # Nombre de los campos
    for archivo in archivos:
        if archivo.endswith('.csv'):
            with open(os.path.join(folder_path, archivo), 'r') as csv_file:
                csv_reader = csv.reader(csv_file, delimiter=';')
                # Saltar la primera fila que contiene los encabezados
                next(csv_reader)
                tipo = archivo.split('_')[0]
                subtipo = archivo.split('.')[0].split('_')[-2:]
                campos_finales.append(f'{tipo}_{subtipo}')
                    
                # Iterar sobre cada fila en el archivo CSV
                for row in csv_reader:
                    serie = row[0] if row else ""
                    codigo_municipio = row[1] if len(row) > 1 else ""
                    nombre = row[2] if len(row) > 2 else ""
                    
                    # Verificar si la serie, código de municipio y nombre ya están en el diccionario
                    if (serie, codigo_municipio, nombre) not in datos_unidos:
                        datos_unidos[(serie, codigo_municipio, nombre)] = {}

                    # Obtener los datos de cada año para el intervalo de edad actual
                    for i, valor in enumerate(row[3:6], start=2020):
                        if 'ss' in subtipo:
                            clave = f"{tipo}_{subtipo[1]}_{i}"
                        else:
                            clave = f"{tipo}_{subtipo[0]}_{subtipo[1]}_{i}"

                        # Agregar los datos al diccionario enlazado
                        if tipo == 'porcentaje':
                            datos_unidos[(serie, codigo_municipio, nombre)][clave] = float(valor) if valor else 0
                        else:
                            datos_unidos[(serie, codigo_municipio, nombre)][clave] = int(valor) if valor else 0

    # Escribir los datos unidos en un nuevo archivo CSV
    with open(output_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=';')
        # Escribir los encabezados
        claves_totales = []

        for key, values in datos_unidos.items():
            if claves_totales == []:
                claves_totales.extend(values.keys())
        claves_totales = [str(clave) for clave in claves_totales]

        # Escribir los encabezados en el archivo CSV
        csv_writer.writerow(['Serie', 'Codigo_municipio', 'Nombre', *claves_totales])

            # Escribir los datos sumados
        for key, values in datos_unidos.items():
            # Crear una lista de valores para cada clave en el orden de las claves totales
            fila = [key[0], key[1], key[2]]
            for clave in claves_totales:
                fila.append(values.get(clave, 0))
            csv_writer.writerow(fila)

# Carpeta donde se encuentran los archivos
folder_path = './clean_data'
# Ruta del archivo CSV de salida
output_path = '../union_final/afiliados_ss.csv'

# Llamar a la función para unir los datos
unir_datos(folder_path, output_path)

